In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import supabase_client
from src.update_pipeline import update_training_week
from src.types.update_pipeline import ExeType
from src import auth_manager
from src.utils import datetime_now_est, get_last_sunday

import os

# New Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.NEW_WEEK, dt=get_last_sunday())

# Mid Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.MID_WEEK, dt=datetime_now_est())

# gen_training_plan

In [7]:
import os
from pprint import pprint

from src.utils import datetime_now_est
from src import supabase_client, activities, training_plan, auth_manager

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
strava_client = auth_manager.get_strava_client(user.athlete_id)
daily_activity = activities.get_daily_activity(strava_client, dt=datetime_now_est(), num_weeks=52)
weekly_summaries = activities.get_weekly_summaries(daily_activity=daily_activity)
resp = await training_plan.gen_training_plan(user, weekly_summaries, dt=datetime_now_est())

pprint(resp.training_plan_weeks)